In [1]:
import json
import numpy as np
import pandas as pd
from collections import Counter

from nltk import word_tokenize, sent_tokenize

from tqdm.auto import tqdm
tqdm.pandas()

pd.set_option('display.max_colwidth', 200)

In [2]:
with open("../data/href2synopsis.json", encoding="utf-8") as f:
    href2synopsis = json.load(f)

    Takeaways:
        - replace \n or \r sequences with single \n
        - use language-specific tokenizer

# BOW stats
    # TODO: add actual bow stats

In [72]:
data = pd.DataFrame()

data["href"] = href2synopsis.keys()
data["synopsis"] = href2synopsis.values()

In [73]:
data = data[data["synopsis"] != ""]

In [74]:
import regex as re

data["synopsis"] = data["synopsis"].apply(
    lambda x: re.sub("\s*\n\s*", "\n", x).strip()
)

In [75]:
# data["min_ord"] = data["synopsis"].progress_apply(
#     lambda s: min([ord(char) for char in s])
# )

# data["max_ord"] = data["synopsis"].progress_apply(
#     lambda s: max([ord(char) for char in s])
# )

# data["mean_ord"] = data["synopsis"].progress_apply(
#     lambda s: np.mean([ord(char) for char in s])
# )

# data["median_ord"] = data["synopsis"].progress_apply(
#     lambda s: np.median([ord(char) for char in s])
# )

# data["mode_ord"] = data["synopsis"].progress_apply(
#     lambda s: Counter([ord(char) for char in s]).most_common()[0][0]
# )

In [84]:
# data["mode_count"] = data["synopsis"].progress_apply(
#     lambda s: Counter(s).most_common()[0][1]
# )

# data["mode_ratio"] = data["synopsis"].progress_apply(
#     lambda s: Counter(s).most_common()[0][1]/len(s)
# )

# data["mean_char_count"] = data["synopsis"].progress_apply(
#     lambda s: np.mean(list(Counter(s).values()))
# )

data["mean_token_count"] = data["synopsis"].progress_apply(
    lambda s: np.mean(list(Counter([w.lower() for w in word_tokenize(s)]).values()))
)

  0%|          | 0/18374 [00:00<?, ?it/s]

In [12]:
data["char_total"] = data["synopsis"].progress_apply(len)
data["token_total"] = data["synopsis"].progress_apply(
    lambda x: len(word_tokenize(x))
)
data["sent_total"] = data["synopsis"].progress_apply(
    lambda x: len([sent for line in x.split("\n") for sent in  sent_tokenize(line) if line.strip()])
)

  0%|          | 0/18374 [00:00<?, ?it/s]

  0%|          | 0/18374 [00:00<?, ?it/s]

  0%|          | 0/18374 [00:00<?, ?it/s]

In [27]:
# data["lines_total"] = data["synopsis"].progress_apply(lambda x: len(x.split("\n")))

  0%|          | 0/19393 [00:00<?, ?it/s]

In [5]:
data["token_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in word_tokenize(x)]) if x.strip() else 0
)

data["sent_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in sent_tokenize(x)]) if x.strip() else 0
)

data["line_max_char_len"] = data["synopsis"].progress_apply(
    lambda x: max([len(t) for t in x.split("\n")]) if x.strip() else 0
)

  0%|          | 0/19393 [00:00<?, ?it/s]

  0%|          | 0/19393 [00:00<?, ?it/s]

  0%|          | 0/19393 [00:00<?, ?it/s]

In [16]:
data["sent_char_ratio"] = [t.sent_total/t.char_total for t in data.itertuples()]
data["sent_token_ratio"] = [t.sent_total/t.token_total for t in data.itertuples()]
data["token_char_ratio"] = [t.token_total/t.char_total for t in data.itertuples()]
data["token_sent_ratio"] = [t.token_total/t.sent_total for t in data.itertuples()]
data["char_sent_ratio"] = [t.char_total/t.sent_total for t in data.itertuples()]
data["char_token_ratio"] = [t.char_total/t.token_total for t in data.itertuples()]

In [13]:
data.describe().astype(int)

,char_total,token_total,sent_total
count,18374,18374,18374
mean,519,105,5
std,348,70,3
min,1,1,1
25%,262,53,3
50%,454,92,5
75%,700,141,7
max,5799,1201,79


In [85]:
stats = data.drop(columns=["href", "synopsis"]) #.drop(columns=["line_max_char_len"])

# Outlier detection

[The svm.OneClassSVM is known to be sensitive to outliers and thus does not perform very well for outlier detection](https://scikit-learn.org/stable/modules/outlier_detection.html#outlier-detection)

In [86]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

In [87]:
clf = LocalOutlierFactor(n_neighbors=20)
preds = clf.fit_predict(stats)

data["pred"] = preds
data["negative_outlier_factor_"] = clf.negative_outlier_factor_

In [88]:
# data[data["pred"] == -1]["synopsis"].sample(10).to_list()

In [89]:
data[data["pred"] == -1].sort_values(by="negative_outlier_factor_"
                                    )[['href', 'synopsis', 'pred', 'negative_outlier_factor_']].head(20)

,href,synopsis,pred,negative_outlier_factor_
1398,/71913-stalker,"Discover the secret of a nerdy young man who falls in love with a third-year senior. Because his dream is so far from reach, observing is the only thing he could do. “Kaojao told me I was falling ...",-1,-1.027231e+06
5458,/532-ryoma-den,"The 49th NHK Taiga drama is Ryoma den, set back in the Bakumatsu period. The story focuses on the life of Sakamoto Ryoma, who lived from 1835 to 1867. He was an important leader in the movement to...",-1,-9.789186e+05
10896,/706775-growing-pain,"A program focusing on retro and feelings. Guests are randomly divided into two lineups for PK in each issue to meet the challenge in the form of ""opening a blind box"". The challenge content includ...",-1,-9.789186e+05
8204,/70421-wayvision,"The WayV members are next up to star in their very own 'Seezn' reality series, 'WayVision'! This marks WayV's first ever reality show to air in Korea since the group's debut. The series' title 'Wa...",-1,-9.789186e+05
3551,/28202-omniscient-interfering-view,"The managers, who know more about the stars they are working for, have unsealed their lips! In the program, the show’s hosts receive detailed descriptions of celebrities’ lifestyles and “interfere...",-1,-7.499255e+05
18185,/59115-mysterious-nile-girl-thutmose,"Sanae visits her ancestors' grave to report she was moved house. However, her arrival damages the family mausoleum, and releases 51 ancient spirits of the Nile that were formerly sealed inside. Sa...",-1,-7.499255e+05
5713,/14183-l.o.r.d,"The story takes place in a sacred mainland of Odin (or Wotan), separated into four counties based on the four elements - water, fire, wind and earth. The published book(s) in the series mainly foc...",-1,-7.499255e+05
13474,/681035-yi-lu-chao-yang,"~~ Adapted from the novel ""Da Cheng Xiao Shi"" (大城小室) by Jiang Li Han (姜立涵).",-1,-7.044481e+05
16641,/703595-my-heroic-husband-season-2,"~~ Adapted from the novel ""Zhui Xu"" (赘婿) by Fen Nu de Xiang Jiao (愤怒的香蕉).",-1,-6.981945e+05
2562,/23122-in-between-seasons,"Mi Kyung lives apart from her husband and raises Soo Hyun, her high school-aged son. One day, Soo Hyun brings home his friend Yong Joon to stay. A few years later, after completing his military se...",-1,-6.463988e+05
